In [1]:
import pandas as pd
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
# 1 政治 
# 17 財經
# 2 國際
# 6 社會
# 9 影劇
# 10 體育
# 20 3c
# 30 時尚 
# 24 遊戲
# 5 生活

text = []
for i in [1,17,2,6]:
    path = "/kaggle/input/5555222266669999/"+str(i)+".txt"
    f = open(path)
    for line in f:
        text.append(line)    
    f.close()
print(text[0])

台北報導，台北市士林區發生一家三口（案18375、案18396、案18441）確診，而案18375的新北市友人（案18397）、新北市客戶（案18518）也染疫；外傳案18518為雜誌編輯，曾經到過時尚品牌Louis Vuitton 21日舉辦男裝店活動。對此，台北市衛生局證實，目前正在匡列疫調當中，且「足跡很多」。，記者黃克翔攝），北市士林區一家三口染疫，目前感染源不明，已知分別為在南港房仲業的哥哥（案18375）於19日發燒、22日快篩陽性，其同住妹妹（案18396）於21日有症狀，22日快篩陽性，兩人分別到醫院做PCR確診，因此媽媽（案18441）匡列採檢後染疫。，此外，案18375的新北市友人（案18397）也於17日有喉嚨癢、咳嗽、流鼻水、肌肉痠痛的情況，並且在23日確診；案18375的新北市客戶（案18518）為50多歲男性，23日遭匡列隔離，24日採檢PCR陽性確診，Ct值19.4。，外傳案18518為一名雜誌編輯，曾經有參與LV於21日在台北101舉辦的男裝店活動，當天除了有大量媒體到場外，還有邀請藝人陳柏霖、劉冠廷、林柏宏、林哲熹、范少勳、鞍馬王子李智凱等人。，針對LV活動傳出確診者足跡，台北市衛生局表示，目前還在匡列、疫調中，確實是新北市案18518的足跡，現在正在匡列當中，因為同仁於下班才接到外縣市資料，足跡很多，且不只一次到過台北101，必須26日才能逐一說明。，指揮中心提供），



In [3]:
 len(text)

3887

In [4]:
import re
listx=[]
lists=[]

index=0
now=0
templong=0
t=1
for y in text:
  y = re.sub("[A-Za-z0-9]", "", y)
  for x in y: 
    t=1
    n="""，。！？｡＂,＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
    p="""｡。，！"""
    q = "? " 
    w = "! " 
    e = "，"
    f = "."
    v = "\n"
    if(x not in p and x not in n and x!="\u3000" and x!=q and x!=w and x!=e and x!=f and x!=v):
      listx.append(x)
      lists.append('Sentence: '+str(now))
      templong+=1
    if(x in p or x =="\u3000"or x==q or x==w or x==e or x==f or x=="\n") :
      now+=1
      t=0
  if(t==1):
      now+=1
    
print(now)

197770


In [5]:
 len(text)

3887

In [6]:
data = pd.DataFrame(list(zip(lists,listx)),columns =['Sentence #','Word'])
data[:50]

,Sentence #,Word
0,Sentence: 0,台
1,Sentence: 0,北
2,Sentence: 0,報
3,Sentence: 0,導
4,Sentence: 1,台
5,Sentence: 1,北
6,Sentence: 1,市
7,Sentence: 1,士
8,Sentence: 1,林
9,Sentence: 1,區


In [7]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        self.grouped = self.data.groupby("Sentence #").apply(self.func)
        self.sentences = [s for s in self.grouped]

    def func(self,w):
      ww=""
      for www in w["Word"].values.tolist():
        ww+=www
        ww+=" "
      return ww


In [8]:
getter = SentenceGetter(data)
sentences = getter.sentences
sentences[:300]

['台 北 報 導 ',
 '台 北 市 士 林 區 發 生 一 家 三 口 案 案 案 確 診 ',
 '北 市 士 林 區 一 家 三 口 染 疫 ',
 '個 案 職 場 位 在 北 市 ',
 '包 括 現 任 行 政 院 長 蘇 貞 昌 駐 日 大 使 謝 長 廷 ',
 '莊 瑞 雄 無 奈 表 示 ',
 '除 了 三 波 增 資 外 ',
 '也 將 在 年 底 前 引 進 策 略 投 資 人 ',
 '市 場 人 士 透 露 ',
 '已 有 組 以 上 買 家 對 三 商 美 邦 人 壽 進 行 實 地 查 核 ',
 '近 期 甚 至 已 進 入 議 價 階 段 ',
 '向 三 商 美 邦 人 壽 求 證 ',
 '公 司 方 面 僅 表 示 交 易 接 洽 事 宜 正 在 進 行 中 ',
 '無 法 公 布 相 關 細 節 ',
 '他 自 己 是 好 人 ',
 '金 融 圈 預 期 ',
 '若 出 價 能 落 在 三 商 壽 大 股 東 設 定 範 圍 ',
 '最 快 年 底 前 即 可 拍 定 ',
 '公 司 財 務 危 機 也 可 暫 解 ',
 '創 辦 人 翁 肇 喜 回 鍋 穩 軍 心 一 段 時 日 ',
 '先 前 被 問 起 不 少 金 控 對 三 商 壽 很 感 興 趣 ',
 '翁 肇 喜 坦 言 ',
 '並 沒 有 把 這 當 作 最 優 先 的 考 慮 ',
 '第 一 優 先 是 把 現 在 公 開 募 資 做 好 ',
 '也 很 正 派 ',
 '好 好 找 到 策 略 投 資 人 ',
 '期 盼 這 個 策 略 投 資 人 不 僅 僅 是 給 予 資 金 ',
 '也 希 望 能 帶 來 新 的 經 營 觀 念 ',
 '之 後 再 慢 慢 考 慮 其 他 方 式 ',
 '翁 肇 喜 對 三 商 壽 的 嫁 妝 很 有 信 心 ',
 '近 期 因 為 升 息 ',
 '部 分 壽 險 公 司 出 狀 況 ',
 '我 們 不 比 他 們 差 ',
 '三 商 水 ',
 '只 是 沒 用 廣 告 去 呈 現 而 已 ',
 '跟 國 泰 富 邦 比 起 來 ',
 '資 產 也 有 到 ',
 '兆 ',
 '這 就 是 嫁 妝 ',
 '且 我 們 業 務 團 隊 非 常 強 ',
 '

In [9]:
sentences[0]

'台 北 報 導 '

In [10]:
len(sentences[0])

8

In [11]:
len(sentences)

171895

In [12]:
# getter.grouped.size()

In [13]:
# getter.grouped.get_group("Sentence: 0")

In [14]:
for h in range(3):
    for s in sentences:
      if(len(s)<=10):
        sentences.remove(s)
    len(sentences)

In [15]:
from pandas.io.parsers.base_parser import is_index_col
ss=[]
m=0
ii=0
si=0
b=40
bb=0
bbb=0
for s in sentences: 
  ii+=1
  # if ii==34808 or ii==229743:
    # continue
  if(len(s)>m):
    m=len(s)
    si=ii
  if(len(s)>b):
    bb+=1
  if(len(s)<=10):
    bbb+=1
  ss.append(len(s))

ss=np.array(ss)
print(np.mean(ss))
print(np.max(ss))
print(np.min(ss))
# print(si)
print(bb)
print(bbb)

26.74149207792662
246
2
17188
88


In [16]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sentence_lengths = sentences["s"].value_counts()
# _, ax = plt.subplots(figsize=(10, 6))
# ax.set_xlabel("Length (in word)")
# sns.histplot(data=sentence_lengths, ax=ax)
# ax.axvline(sentence_lengths.mean(), color='red', linestyle='dashed', linewidth=1);

In [17]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print("Total number of words: ", total_words)

Total number of words:  4797


In [18]:
input_sequences = []
i=0
for line in sentences:
    if(i==0):
        print(line)
    token_list = tokenizer.texts_to_sequences([line])[0]
    if(i==0):
        print(token_list)
        i+=1
#     j=0
    for i in range(1, len(token_list)):
#         if(i>5):
#             j+=1
        input_sequences.append(token_list[:i+1])
    
print("Total input sequences: ", len(input_sequences))

台 北 市 士 林 區 發 生 一 家 三 口 案 案 案 確 診 
[15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248, 247, 103, 103, 103, 225, 418]
Total input sequences:  1757292


In [19]:
# tokenizer.word_index

In [20]:
import json
with open('word.json', 'w', encoding='utf-8') as f:
  json.dump(tokenizer.word_index, f)

In [21]:
input_sequences[:50]

[[15, 76],
 [15, 76, 23],
 [15, 76, 23, 386],
 [15, 76, 23, 386, 150],
 [15, 76, 23, 386, 150, 161],
 [15, 76, 23, 386, 150, 161, 21],
 [15, 76, 23, 386, 150, 161, 21, 43],
 [15, 76, 23, 386, 150, 161, 21, 43, 4],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248, 247],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248, 247, 103],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248, 247, 103, 103],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248, 247, 103, 103, 103],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248, 247, 103, 103, 103, 225],
 [15, 76, 23, 386, 150, 161, 21, 43, 4, 37, 248, 247, 103, 103, 103, 225, 418],
 [76, 23],
 [76, 23, 386],
 [76, 23, 386, 150],
 [76, 23, 386, 150, 161],
 [76, 23, 386, 150, 161, 4],
 [76, 23, 386, 150, 161, 4, 37],
 [76, 23, 386, 150, 161, 4, 37, 248],
 [76, 23, 386, 150, 161, 4, 37, 248, 247],
 [76, 23, 386, 150, 161, 4, 37, 248, 247, 607],
 [76, 23, 386, 150, 1

In [22]:
max_sequence_len=20
input_sequences = tf.keras .preprocessing.sequence .pad_sequences(input_sequences, maxlen=max_sequence_len)

In [23]:
# input_sequences[:20]

In [24]:
input_sequences=np.array(input_sequences)
len(input_sequences)

1757292

In [25]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [26]:
len(ys)

1757292

In [27]:
# xs[:10]

In [28]:
# ys[:10]

In [29]:
print(total_words)
print(max_sequence_len)
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())
print(model)

# model.fit(xs, ys, epochs=15, verbose=1)

4797
20


2023-01-09 13:02:57.419809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 13:02:57.420785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 13:02:57.559244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 13:02:57.560188: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 13:02:57.561070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 100)           479700    
_________________________________________________________________
bidirectional (Bidirectional (None, 300)               301200    
_________________________________________________________________
dense (Dense)                (None, 4797)              1443897   
Total params: 2,224,797
Trainable params: 2,224,797
Non-trainable params: 0
_________________________________________________________________
None


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [30]:
# (epoch to start, learning rate) tuples
LR_SCHEDULE = [(5, 0.0005), (10, 0.0001)]

def lr_schedule(epoch, lr):
  """Helper function to retrieve the scheduled learning rate based on epoch."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr

In [31]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Get the current learning rate from model's optimizer.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # Call schedule function to get the scheduled learning rate.
    scheduled_lr = self.schedule(epoch, lr)
    # Set the value back to the optimizer before this epoch starts
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('Epoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr))

In [32]:
def data_generator(data, targets, batch_size):
    batches = (len(data) + batch_size - 1)//batch_size
    while(True):
         for i in range(batches):
              X = data[i*batch_size : (i+1)*batch_size]
              Y = targets[i*batch_size : (i+1)*batch_size]
              yield (X, Y)

In [33]:
batch_size=32
num_epochs=12
model.fit_generator(generator = data_generator(xs, ys, batch_size),
                    steps_per_epoch = (len(data) + batch_size - 1) // batch_size,
                    epochs = num_epochs,
                    verbose = 1,
                    callbacks=[LearningRateScheduler(lr_schedule)])

model.save_weights('new_weights.h5')

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2023-01-09 13:03:01.872261: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/12

Epoch 00000: Learning rate is 0.0010.


2023-01-09 13:03:05.489904: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


62984/62984 [==============================] - 614s 10ms/step - loss: 4.8938 - accuracy: 0.2290
Epoch 2/12

Epoch 00001: Learning rate is 0.0010.
62984/62984 [==============================] - 586s 9ms/step - loss: 4.3648 - accuracy: 0.2779
Epoch 3/12

Epoch 00002: Learning rate is 0.0010.
62984/62984 [==============================] - 579s 9ms/step - loss: 4.2091 - accuracy: 0.2909
Epoch 4/12

Epoch 00003: Learning rate is 0.0010.
62984/62984 [==============================] - 573s 9ms/step - loss: 4.1055 - accuracy: 0.3013
Epoch 5/12

Epoch 00004: Learning rate is 0.0010.
62984/62984 [==============================] - 570s 9ms/step - loss: 4.0228 - accuracy: 0.3109
Epoch 6/12

Epoch 00005: Learning rate is 0.0005.
62984/62984 [==============================] - 564s 9ms/step - loss: 3.9560 - accuracy: 0.3141
Epoch 7/12

Epoch 00006: Learning rate is 0.0005.
62984/62984 [==============================] - 558s 9ms/step - loss: 3.8691 - accuracy: 0.3249
Epoch 8/12

Epoch 00007: Learning 

In [34]:
text_all=['蔡 英',
 '不 要 期',
 '我 跟 我 朋',
 '慶 祝 紀 念',
 '今',
 '白 天 穿',
 '資 訊',
 '今 年 暑',
 '官 方 推',
 '外 籍 投',
 '議 員 發 出 聲',
 '今 天 早']
output=[]
for i in range(len(text_all)):
    seed_text = text_all[i]
    next_words = 20

    for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted = model.predict(token_list)
      predicted = np.argmax(predicted[0], axis=-1)
      index_to_label = {v: k for k, v in tokenizer.word_index.items()}
      t= index_to_label[predicted]
      seed_text += " " + t
    print(seed_text)
    output.append(seed_text)
# print(output)

蔡 英 文 也 表 示 自 己 的 民 眾 黨 團 也 會 在 年 底 台 北 市 議
不 要 期 待 在 月 日 至 月 日 至 月 日 止 日 前 開 始 進 入 第 四 季
我 跟 我 朋 友 也 要 做 好 好 好 的 人 都 不 會 有 什 麼 事 情 都 不 會
慶 祝 紀 念 日 前 首 度 參 選 人 陳 時 中 日 前 在 臉 書 發 文 撤 回 回
今 年 月 日 至 月 日 止 日 前 將 在 月 日 起 開 始 增 加 新 北
白 天 穿 著 一 套 瓶 頸 要 求 大 家 看 到 的 衣 服 也 不 用 的 朋 友
資 訊 網 路 購 買 的 獎 金 就 是 一 個 好 的 人 選 人 不 要 讓 人
今 年 暑 假 期 間 就 會 延 續 時 間 點 多 為 年 以 下 有 期 徒 刑 第
官 方 推 出 新 竹 市 長 候 選 人 張 國 明 的 人 員 研 究 委 員 會 一
外 籍 投 資 人 員 在 年 月 日 前 接 受 媒 體 聯 訪 時 表 示 媒 體 報
議 員 發 出 聲 明 表 示 不 要 再 次 出 席 台 北 市 長 候 選 人 黃 珊 珊 見
今 天 早 上 點 多 時 間 為 確 診 者 家 人 確 診 數 為 萬 人 的 人 數


In [35]:
batch_size=32
num_epochs=5

LR_SCHEDULE = [(0, 0.0001), (3, 0.00005)]

model.fit_generator(generator = data_generator(xs, ys, batch_size),
                    steps_per_epoch = (len(data) + batch_size - 1) // batch_size,
                    epochs = num_epochs,
                    verbose = 1,
                    callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/5

Epoch 00000: Learning rate is 0.0001.
62984/62984 [==============================] - 562s 9ms/step - loss: 3.6738 - accuracy: 0.3471
Epoch 2/5

Epoch 00001: Learning rate is 0.0001.
62984/62984 [==============================] - 554s 9ms/step - loss: 3.6753 - accuracy: 0.3470
Epoch 3/5

Epoch 00002: Learning rate is 0.0001.
62984/62984 [==============================] - 555s 9ms/step - loss: 3.6707 - accuracy: 0.3471
Epoch 4/5

Epoch 00003: Learning rate is 0.0001.
62984/62984 [==============================] - 559s 9ms/step - loss: 3.6749 - accuracy: 0.3468
Epoch 5/5

Epoch 00004: Learning rate is 0.0000.
62984/62984 [==============================] - 558s 9ms/step - loss: 3.6393 - accuracy: 0.3520


In [36]:
text_all=['蔡 英',
 '不 要 期',
 '我 跟 我 朋',
 '慶 祝 紀 念',
 '今',
 '白 天 穿',
 '資 訊',
 '今 年 暑',
 '官 方 推',
 '外 籍 投',
 '議 員 發 出 聲',
 '今 天 早']
output=[]
for i in range(len(text_all)):
    seed_text = text_all[i]
    next_words = 20

    for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted = model.predict(token_list)
      predicted = np.argmax(predicted[0], axis=-1)
      index_to_label = {v: k for k, v in tokenizer.word_index.items()}
      t= index_to_label[predicted]
      seed_text += " " + t
    print(seed_text)
    output.append(seed_text)
# print(output)

蔡 英 文 也 表 示 自 己 的 民 主 黨 籍 參 選 人 蔣 萬 安 們 不 會
不 要 期 待 在 台 灣 的 國 家 的 國 際 社 會 的 民 主 黨 主 席 朱 立
我 跟 我 朋 友 的 人 都 不 會 有 一 個 人 的 朋 友 都 是 有 什 麼 事 情
慶 祝 紀 念 日 的 時 候 也 是 一 個 黨 的 蔣 萬 安 的 蔣 經 國 民 黨 的
今 年 月 日 至 月 日 至 月 日 至 月 日 至 月 日 至 月 日 至 日
白 天 穿 著 一 條 的 黑 衣 人 員 在 臉 書 上 看 到 陳 時 中 的 澄 清
資 訊 媒 體 人 員 也 指 出 台 灣 的 立 場 日 前 發 布 新 聞 稿 表
今 年 暑 假 期 間 時 間 點 分 到 了 一 個 月 的 時 間 點 分 時 間 時
官 方 推 出 的 時 候 選 人 不 會 有 任 何 人 選 人 的 影 響 力 量 會
外 籍 投 人 人 士 也 在 臉 書 發 文 表 示 不 要 再 度 出 現 了 解 他
議 員 發 出 聲 明 表 示 不 要 讓 人 民 感 到 不 滿 意 的 事 情 是 不 是 一
今 天 早 上 點 多 時 間 為 陽 性 確 診 案 例 確 診 案 例 的 陰 算 案


In [37]:
batch_size=32
num_epochs=5

LR_SCHEDULE = [(0, 0.00005), (2, 0.00003)]

model.fit_generator(generator = data_generator(xs, ys, batch_size),
                    steps_per_epoch = (len(data) + batch_size - 1) // batch_size,
                    epochs = num_epochs,
                    verbose = 1,
                    callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/5

Epoch 00000: Learning rate is 0.0001.
62984/62984 [==============================] - 563s 9ms/step - loss: 3.6408 - accuracy: 0.3520
Epoch 2/5

Epoch 00001: Learning rate is 0.0000.
62984/62984 [==============================] - 552s 9ms/step - loss: 3.6467 - accuracy: 0.3513
Epoch 3/5

Epoch 00002: Learning rate is 0.0000.
62984/62984 [==============================] - 551s 9ms/step - loss: 3.6648 - accuracy: 0.3485
Epoch 4/5

Epoch 00003: Learning rate is 0.0000.
62984/62984 [==============================] - 561s 9ms/step - loss: 3.6582 - accuracy: 0.3500
Epoch 5/5

Epoch 00004: Learning rate is 0.0000.
62984/62984 [==============================] - 548s 9ms/step - loss: 3.6288 - accuracy: 0.3544


In [38]:
text_all=['蔡 英',
 '不 要 期',
 '我 跟 我 朋',
 '慶 祝 紀 念',
 '今',
 '白 天 穿',
 '資 訊',
 '今 年 暑',
 '官 方 推',
 '外 籍 投',
 '議 員 發 出 聲',
 '今 天 早']
output=[]
for i in range(len(text_all)):
    seed_text = text_all[i]
    next_words = 20

    for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted = model.predict(token_list)
      predicted = np.argmax(predicted[0], axis=-1)
      index_to_label = {v: k for k, v in tokenizer.word_index.items()}
      t= index_to_label[predicted]
      seed_text += " " + t
    print(seed_text)
    output.append(seed_text)
# print(output)

蔡 英 文 在 台 北 市 政 府 提 供 的 人 選 的 時 候 選 情 勢 不 會
不 要 期 待 在 台 灣 的 國 家 的 國 際 社 會 的 民 眾 都 是 不 是 不
我 跟 我 朋 友 的 人 都 不 會 有 一 個 人 的 朋 友 都 是 有 什 麼 事 情
慶 祝 紀 念 日 的 時 候 也 是 一 個 黨 的 蔣 萬 安 的 蔣 經 國 民 黨 的
今 天 下 午 時 許 台 北 市 長 柯 文 哲 日 前 接 受 媒 體 人 說
白 天 穿 著 衣 服 用 的 衣 服 用 來 看 看 看 法 的 人 不 見 看 好 的
資 訊 媒 體 人 員 也 指 出 台 灣 的 立 場 日 前 發 布 新 聞 稿 指
今 年 暑 假 期 間 時 間 點 分 左 右 後 來 到 場 所 有 人 才 發 現 時
官 方 推 出 新 北 市 長 侯 友 宜 競 選 總 部 提 供 的 人 喊 話 題 的
外 籍 投 人 人 士 也 在 臉 書 發 文 表 示 不 要 錯 假 的 言 論 戰 爭
議 員 發 出 聲 明 表 示 不 會 有 人 不 捨 不 到 的 陳 時 中 就 是 不 是 不
今 天 早 上 點 多 時 間 為 陽 性 確 診 案 例 的 確 診 案 例 採 檢 陽
